<a href="https://colab.research.google.com/github/vadigr123/colab-testing/blob/main/video_to_frames_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ### 🖼️ Choose frame image format
frame_format = "png"  #@param ["png", "jpg", "jpeg", "bmp", "tiff", "webp"] {type:"string"}

#@markdown ### 📦 Upload a ZIP file containing video files (.mp4, .webm, etc.)
from google.colab import files
import os
import zipfile
import shutil
import glob

# Upload ZIP file
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]

# Extract the uploaded ZIP
extract_dir = "videos"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ ZIP extracted!")

# Frame output directory
frames_root = "frames_output"
os.makedirs(frames_root, exist_ok=True)

# Supported video formats
supported_exts = [".mp4", ".webm", ".mov", ".mkv", ".avi"]

# Find video files
video_files = glob.glob(f"{extract_dir}/*")
video_files = [f for f in video_files if os.path.splitext(f)[1].lower() in supported_exts]

if not video_files:
    print("⚠️ No supported video files found.")
else:
    for video_file in video_files:
        filename = os.path.splitext(os.path.basename(video_file))[0]
        out_folder = os.path.join(frames_root, filename)
        os.makedirs(out_folder, exist_ok=True)

        print(f"🎞️ Extracting frames from {filename}...")

        # FFmpeg command to extract frames
        !ffmpeg -i "{video_file}" "{out_folder}/frame_%04d.{frame_format}" -hide_banner -loglevel error

# Create final ZIP with all frame folders
final_zip = "extracted-frames.zip"
shutil.make_archive("extracted-frames", 'zip', frames_root)

# Download final zip
files.download(final_zip)

print("✅ Done!")


In [ ]:
#@markdown ### 🖼️ Generate a horizontal grid of first N frames per folder and create a downloadable ZIP
from PIL import Image
import os
import glob
import zipfile

#@markdown Select how many frames to include from each folder:
num_frames = 4 #@param ["1", "2", "3", "4", "5", "6", "8", "10"] {type:"raw"}

frame_dir = "frames_output"  # Folder with subfolders containing frames
output_grid_zip = "all_grids_preview.zip"
temp_grids_dir = "grids_temp"

os.makedirs(temp_grids_dir, exist_ok=True)

folders = sorted(os.listdir(frame_dir))

for folder in folders:
    folder_path = os.path.join(frame_dir, folder)
    if not os.path.isdir(folder_path): continue

    # Get frame images
    frame_paths = sorted(glob.glob(os.path.join(folder_path, "frame_*.png")))[:num_frames]
    if not frame_paths:
        print(f"⚠️ No frames found in {folder}")
        continue

    # Load images
    images = [Image.open(f).convert("RGBA") for f in frame_paths]

    # Match height of all frames
    min_height = min(img.height for img in images)
    resized = [
        img.resize((int(img.width * min_height / img.height), min_height), Image.LANCZOS)
        for img in images
    ]

    # Combine horizontally
    total_width = sum(img.width for img in resized)
    combined_image = Image.new("RGBA", (total_width, min_height), (255, 255, 255, 0))
    x_offset = 0
    for img in resized:
        combined_image.paste(img, (x_offset, 0))
        x_offset += img.width

    # Save to temp directory
    output_path = os.path.join(temp_grids_dir, f"{folder}_grid.png")
    combined_image.save(output_path)
    print(f"✅ Created: {output_path}")

# Zip all generated grids
with zipfile.ZipFile(output_grid_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file_name in sorted(os.listdir(temp_grids_dir)):
        file_path = os.path.join(temp_grids_dir, file_name)
        zipf.write(file_path, arcname=file_name)

print(f"\n✅ All grid previews saved to ZIP: {output_grid_zip}")

# Provide download link
from google.colab import files
files.download(output_grid_zip)
